In [142]:
import numpy as np
import pandas as pd
import re
import time

# from sklearn.impute import SimpleImputer
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
import tensorflow as tf

In [4]:
df = pd.read_csv("data/train.csv")

df.head()

f:\RaifhackDS2021\venv\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
0,Пермь,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,...,5.762963,5.530612,1964.118519,1960.959184,Пермский край,32.0,S27289,2020-01-05,10,0
1,Шатура,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,...,2.894366,3.527778,1952.321678,1957.222222,Московская область,280.0,S17052,2020-01-05,10,0
2,Ярославль,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,...,6.141414,7.222222,1968.150000,1973.370370,Ярославская область,297.4,S16913,2020-01-05,110,0
3,Новокузнецк,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,...,8.581081,9.000000,1992.716216,2014.000000,Кемеровская область,190.0,S10148,2020-01-05,110,0
4,Москва,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,...,7.263889,5.684211,1963.229167,1960.500000,Москва,60.2,S1338,2020-01-05,10,0


In [5]:
def parse_floor(floor):
    try:
        return str(int(float(floor)))
    except Exception as e:
        if isinstance(floor, str):
            splits = [x.strip() for x in floor.split(',') if x.strip() != '']
            if len(splits) == 1:
                if re.match("[0-9]\s*-\s*[0-9]", floor):
                    return "multifloors"
                if re.match("[0-9]\s*-\s*\D", floor):
                    return floor.split('-')[0].strip()
                if "+" in floor:
                    return "multifloors"
                return str(floor)
            else:
                return "multifloors"
        if np.isnan(floor):
            return "nan"

In [6]:
df_targets = df['per_square_meter_price']
df_train = df.drop(columns=[
    "id", "date", 'per_square_meter_price'
])
df_train["floor"] = df_train['floor'].map(lambda x: parse_floor(x))
df_train['street'] = df_train['street'].fillna('nan')
df_train

,city,floor,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,...,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,realty_type,price_type
0,Пермь,nan,57.998207,56.292797,4,19,35,52,0,0,...,765.0,5.762963,5.530612,1964.118519,1960.959184,Пермский край,32.000000,S27289,10,0
1,Шатура,nan,55.574284,39.543835,3,24,37,59,0,0,...,514.0,2.894366,3.527778,1952.321678,1957.222222,Московская область,280.000000,S17052,10,0
2,Ярославль,nan,57.619140,39.850525,1,30,67,128,0,0,...,573.0,6.141414,7.222222,1968.150000,1973.370370,Ярославская область,297.400000,S16913,110,0
3,Новокузнецк,nan,53.897083,87.108604,0,0,5,21,0,0,...,54.0,8.581081,9.000000,1992.716216,2014.000000,Кемеровская область,190.000000,S10148,110,0
4,Москва,nan,55.802590,37.487110,1,23,64,153,0,1,...,619.0,7.263889,5.684211,1963.229167,1960.500000,Москва,60.200000,S1338,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279787,Томск,1,56.459183,84.979334,2,33,111,222,0,0,...,979.0,5.714286,5.882353,1972.260870,1973.460000,Томская область,358.320073,S11114,10,1
279788,Санкт-Петербург,1,59.936954,30.356383,10,274,718,1340,0,2,...,1659.0,4.719388,4.706667,1876.994898,1873.186667,Санкт-Петербург,119.637556,S28440,110,1
279789,Калининград,3,54.729233,20.514968,0,12,34,84,0,2,...,460.0,3.950413,4.885714,1964.258333,1970.571429,Калининградская область,312.789725,S6671,10,1
279790,Кемерово,1,55.360680,86.081460,5,57,100,134,0,0,...,429.0,4.691489,4.125000,1957.425532,1954.625000,Кемеровская область,89.201305,S17667,110,1


In [18]:
columns_4dim = [
    'osm_catering_points_in_0.001','osm_catering_points_in_0.005','osm_catering_points_in_0.0075','osm_catering_points_in_0.01',
    'osm_shops_points_in_0.001','osm_shops_points_in_0.005','osm_shops_points_in_0.0075','osm_shops_points_in_0.01',
    'osm_offices_points_in_0.001', 'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075', 'osm_offices_points_in_0.01',
    "osm_finance_points_in_0.001","osm_finance_points_in_0.005","osm_finance_points_in_0.0075","osm_finance_points_in_0.01",
    'osm_building_points_in_0.001','osm_building_points_in_0.005','osm_building_points_in_0.0075','osm_building_points_in_0.01',
    'osm_culture_points_in_0.001','osm_culture_points_in_0.005','osm_culture_points_in_0.0075','osm_culture_points_in_0.01',
    'osm_amenity_points_in_0.001','osm_amenity_points_in_0.005','osm_amenity_points_in_0.0075','osm_amenity_points_in_0.01',
    'osm_crossing_points_in_0.001','osm_crossing_points_in_0.005','osm_crossing_points_in_0.0075','osm_crossing_points_in_0.01',
]
df_train_4dim = df_train[columns_4dim]
# df_shops = df[['osm_shops_points_in_0.001', 'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075', 'osm_shops_points_in_0.01']]
# df_offices = df[['osm_offices_points_in_0.001','osm_offices_points_in_0.005','osm_offices_points_in_0.0075','osm_offices_points_in_0.01']]
# df_finances = df[['osm_finance_points_in_0.001','osm_finance_points_in_0.005','osm_finance_points_in_0.0075','osm_finance_points_in_0.01']]
# df_transport_stops = df[['osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005','osm_transport_stop_points_in_0.0075','osm_transport_stop_points_in_0.01']]
# df_building = df[['osm_building_points_in_0.001','osm_building_points_in_0.005','osm_building_points_in_0.0075','osm_building_points_in_0.01']]
# df_culture = df[['osm_culture_points_in_0.001','osm_culture_points_in_0.005','osm_culture_points_in_0.0075','osm_culture_points_in_0.01']]

columns_3dim = [
    'osm_healthcare_points_in_0.005','osm_healthcare_points_in_0.0075','osm_healthcare_points_in_0.01',
    'osm_leisure_points_in_0.005','osm_leisure_points_in_0.0075','osm_leisure_points_in_0.01',
    'osm_historic_points_in_0.005','osm_historic_points_in_0.0075','osm_historic_points_in_0.01',
    'osm_hotels_points_in_0.005','osm_hotels_points_in_0.0075','osm_hotels_points_in_0.01',
    'osm_train_stop_points_in_0.005','osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
    'osm_transport_stop_points_in_0.005','osm_transport_stop_points_in_0.0075','osm_transport_stop_points_in_0.01',
]
df_train_3dim = df_train[columns_3dim]

# df_healthcare = df[['osm_healthcare_points_in_0.005','osm_healthcare_points_in_0.0075','osm_healthcare_points_in_0.01']]
# df_leisure = df[['osm_leisure_points_in_0.005','osm_leisure_points_in_0.0075','osm_leisure_points_in_0.01']]
# df_historic = df[['osm_historic_points_in_0.005','osm_historic_points_in_0.0075','osm_historic_points_in_0.01']]
# df_hotel = df[['osm_hotels_points_in_0.005','osm_hotels_points_in_0.0075','osm_hotels_points_in_0.01']]
# df_train_stops = df[['osm_train_stop_points_in_0.005','osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01']]

columns_2dim = [
    'lat', 'lng',
    'osm_city_nearest_name', 'osm_city_nearest_population',
    'reform_count_of_houses_1000','reform_count_of_houses_500',
    'reform_mean_floor_count_500','reform_mean_floor_count_1000',
    'reform_mean_year_building_500','reform_mean_year_building_1000',
    'reform_house_population_500','reform_house_population_1000',
]
df_train_2dim = df_train[columns_2dim]

# df_closest_stops = df[['osm_subway_closest_dist','osm_train_stop_closest_dist']]
# df_amenity = df[['osm_amenity_points_in_0.001','osm_amenity_points_in_0.005']]
# df_reform_count = df[['reform_count_of_houses_1000', 'reform_count_of_houses_500']]
# df_house_population = df[['reform_house_population_1000', 'reform_house_population_500']]
# df_mean_floor = df[['reform_mean_floor_count_1000', 'reform_mean_floor_count_500']]
# df_year_building = df[['reform_mean_year_building_1000', 'reform_mean_year_building_500']]
# df_pos = df[['lat', 'lng']]

columns_type = ['region', 'city', 'street', 'floor',  'realty_type', 'total_square']
df_train_type = df_train[columns_type]

columns_dists = ['osm_city_closest_dist', 'osm_subway_closest_dist', 'osm_train_stop_closest_dist', 'osm_transport_stop_closest_dist',  'osm_crossing_closest_dist']
df_train_dists = df_train[columns_dists]

columns_pt = ['price_type']
df_train_pt = df_train[columns_pt]

In [19]:
len(set(columns_4dim+columns_2dim+columns_3dim+columns_type+columns_dists+columns_pt))

74

In [22]:
df_train_4dim.shape, df_train_3dim.shape, df_train_2dim.shape, df_train_type.shape, df_train_dists.shape, df_train_pt.shape

((279792, 32),
 (279792, 18),
 (279792, 12),
 (279792, 6),
 (279792, 5),
 (279792, 1))

In [23]:
def transform_data(df):
    features = df.columns

    numeric_features = df._get_numeric_data().columns
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('imputer', SimpleImputer(strategy="constant", fill_value=-1))
    ])

    categorical_features = list(set(features) - set(numeric_features))
    categorical_transformer = Pipeline(steps=[
        ('encoder', OrdinalEncoder()),
        ('imputer', SimpleImputer(strategy="constant", fill_value=-1))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features),
            # ('imputer', SimpleImputer(strategy="constant", fill_value=-1), features)
        ])

    data_transform = Pipeline(steps=[
        ('preprocessor', preprocessor),
    ])

    data = data_transform.fit_transform(df)
    return data


In [24]:
x_train_4dim = transform_data(df_train_4dim)
x_train_3dim = transform_data(df_train_3dim)
x_train_2dim =  transform_data(df_train_2dim)
x_train_type = transform_data(df_train_type)
x_train_dists = transform_data(df_train_dists)
x_train_pt = transform_data(df_train_pt)

In [26]:
len(df_train)

279792

In [64]:
def split_data(data, split_rate=0.7):
    train_size = int(len(data) * split_rate)
    return data[:train_size], data[train_size:]

In [139]:
train_4dim, test_4dim = split_data(x_train_4dim)
train_3dim, test_3dim = split_data(x_train_3dim)
train_2dim, test_2dim = split_data(x_train_2dim)
train_type, test_type = split_data(x_train_type)
train_dists, test_dists = split_data(x_train_dists)
train_pt, test_pt = split_data(x_train_pt)
train_targets, test_targets = split_data(df_targets.values)

train_data = {
    '4dim': train_4dim,
    '3dim': train_3dim,
    '2dim': train_2dim,
    'type': train_type,
    'dists': train_dists,
    'pt': train_pt,
}

test_data = {
    '4dim': test_4dim,
    '3dim': test_3dim,
    '2dim': test_2dim,
    'type': test_type,
    'dists': test_dists,
    'pt': test_pt,
}

In [121]:
tf.realdiv(x_train_4dim.shape[-1], 4)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [231]:
inputs_4dim = tf.keras.Input((x_train_4dim.shape[-1],), name='4dim')
inputs_3dim = tf.keras.Input((x_train_3dim.shape[-1],), name='3dim')
inputs_2dim = tf.keras.Input((x_train_2dim.shape[-1],), name='2dim')
inputs_type = tf.keras.Input((x_train_type.shape[-1],), name='type')
inputs_dists = tf.keras.Input((x_train_dists.shape[-1],), name='dists')
inputs_pt = tf.keras.Input((x_train_pt.shape[-1],), name='pt')

channels_4dim = tf.realdiv(x_train_4dim.shape[-1], 4).numpy()
reshape_4dim = tf.keras.layers.Reshape((4, channels_4dim))(inputs_4dim)
conv_4dim = tf.keras.layers.Conv1D(4, 4, groups=4, activation='relu')(reshape_4dim)
flat_4dim = tf.keras.layers.Flatten()(conv_4dim)

channels_3dim = tf.realdiv(x_train_3dim.shape[-1], 3).numpy()
reshape_3dim = tf.keras.layers.Reshape((3, channels_3dim))(inputs_3dim)
conv_3dim = tf.keras.layers.Conv1D(3, 3, groups=3, activation='relu')(reshape_3dim)
flat_3dim = tf.keras.layers.Flatten()(conv_3dim)

channels_2dim = tf.realdiv(x_train_2dim.shape[-1], 2).numpy()
reshape_2dim = tf.keras.layers.Reshape((2, channels_2dim))(inputs_2dim)
conv_2dim = tf.keras.layers.Conv1D(2, 2, groups=2, activation='relu')(reshape_2dim)
flat_2dim = tf.keras.layers.Flatten()(conv_2dim)

dense_type = tf.keras.layers.Dense(1, activation='relu')(inputs_type)
dense_dists = tf.keras.layers.Dense(1, activation='relu')(inputs_dists)
dense_pt = tf.keras.layers.Dense(1, activation='relu')(inputs_pt)


concat = tf.keras.layers.Concatenate()([flat_4dim, flat_3dim, flat_2dim, dense_type,
 dense_dists,
  dense_pt])
outputs = tf.keras.layers.Dense(1)(concat)

model = tf.keras.Model(
    inputs=[inputs_4dim, inputs_3dim, inputs_2dim, inputs_type, 
        inputs_dists,
        inputs_pt
    ],
    outputs=outputs
)

model.summary()

Model: "functional_32"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
4dim (InputLayer)               [(None, 32)]         0                                            
__________________________________________________________________________________________________
3dim (InputLayer)               [(None, 18)]         0                                            
__________________________________________________________________________________________________
2dim (InputLayer)               [(None, 12)]         0                                            
__________________________________________________________________________________________________
reshape_58 (Reshape)            (None, 4, 8)         0           4dim[0][0]                       
______________________________________________________________________________________

In [236]:
learning_rate = tf.keras.optimizers.schedules.InverseTimeDecay(
    initial_learning_rate=1e-2,
    decay_steps=500,
    decay_rate=0.5
)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.Huber()

model.compile(
    optimizer=optimizer,
    loss=loss
)

In [237]:
@tf.function
def train_step(data, targets):
    with tf.GradientTape() as tape:
        preds = model(data, training=True)
        loss_val = loss(y_true=targets, y_pred=preds)

    grads = tape.gradient(loss_val, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_val

In [238]:
def train(model, x, y, epochs, verbose=1):
    start_time = time.time()
    for epoch in range(epochs):
        optimizer
        loss_val = train_step(x, y)

        if (epoch+1) % verbose == 0 or epoch == 0:
            print(f"Epoch {epoch+1:4d}: loss - {loss_val:.3f} | {time.time() - start_time:.2f}s")
            start_time = time.time()

In [239]:
train(model, train_data, train_targets, epochs=10000, verbose=500)

Epoch    1: loss - 109106.539 | 0.67s
Epoch  500: loss - 100106.227 | 34.05s
Epoch 1000: loss - 89593.844 | 34.12s
Epoch 1500: loss - 87278.852 | 34.18s
Epoch 2000: loss - 86852.016 | 34.10s
Epoch 2500: loss - 86479.242 | 34.08s
Epoch 3000: loss - 86079.359 | 34.14s
Epoch 3500: loss - 85669.016 | 34.07s
Epoch 4000: loss - 85282.859 | 34.11s
Epoch 4500: loss - 84962.297 | 34.07s
Epoch 5000: loss - 84739.883 | 34.13s
Epoch 5500: loss - 84597.672 | 34.05s
Epoch 6000: loss - 84493.320 | 34.10s
Epoch 6500: loss - 84397.211 | 34.10s
Epoch 7000: loss - 84299.039 | 34.12s
Epoch 7500: loss - 84197.156 | 34.14s
Epoch 8000: loss - 84091.344 | 34.12s
Epoch 8500: loss - 83981.078 | 34.14s
Epoch 9000: loss - 83865.445 | 34.12s
Epoch 9500: loss - 83742.078 | 34.18s
Epoch 10000: loss - 83610.625 | 34.20s


In [127]:
def get_deviation(y_pred: float, y_true: float) -> float:
    return (y_true - y_pred) / y_true

def get_hit(deviation: float, w: float=1.1) -> float:
    if deviation < -0.6:
        return 9 * w
    if deviation >= -0.6 and deviation < -0.15:
        return w * ((1 + deviation / 0.15) ** 2)
    if deviation >= -0.15 and deviation < 0.15:
        return 0.0
    if deviation >= 0.15 and deviation < 0.6:
        return (deviation / 0.15 - 1) ** 2
    if deviation >= 0.6:
        return 9.0

def score(y_pred: list, y_true: list, w: float=1.1) -> float:
    y_pred = list(y_pred)
    y_true = list(y_true)
    cum_sum = 0
    for i in range(len(y_pred)):
        dev = get_deviation(y_pred[i], y_true[i])
        cum_sum += get_hit(dev, w)
    return cum_sum / len(y_pred)

In [240]:
preds = model.predict(train_data)
score(preds, train_targets)

array([6.22248014])

In [241]:
preds = model.predict(test_data)
score(preds, test_targets)

array([6.15906644])